
# 🧠 Power BI with Python — Complete Integration and Visualization Tutorial

---

## ⚙️ 1. Overview

**Power BI** is Microsoft’s premier platform for interactive data visualization and analytics.
**Python** enhances Power BI by adding advanced capabilities for:

* Data preprocessing and automation (Power Query)
* Machine learning and forecasting (scikit-learn, statsmodels, prophet)
* Advanced visuals (matplotlib, seaborn, plotly)
* Natural language and statistical modeling (NLP, regression, clustering)
* Integration with external APIs and automation (requests, msal)

When integrated, these two technologies allow for seamless **business intelligence and data science convergence** — from raw data to actionable insight.

---

## 🧩 2. Environment Setup

### Step 1 — Install Required Software

| Component            | Description                        | Source                                         |
| -------------------- | ---------------------------------- | ---------------------------------------------- |
| **Power BI Desktop** | Report design and local testing    | Microsoft Store or Download Center             |
| **Python 3.10+**     | Interpreter for Power BI scripting | [Python.org](https://www.python.org/downloads) |
| **Libraries**        | Core analytical stack              | via `pip install`                              |

```bash
pip install pandas numpy matplotlib seaborn scikit-learn statsmodels prophet plotly psutil
```

### Step 2 — Configure Power BI Desktop

1. In Power BI Desktop, go to
   **File → Options and Settings → Options → Python scripting**
2. Set your **Python home directory**
   Example:
   `C:\Users\<User>\AppData\Local\Programs\Python\Python310`

---

## 📊 3. Using Python in Power BI

### A. Power Query (Data Transform Layer)

You can run Python scripts as part of your **data import pipeline**.

**Steps:**

1. **Home → Get Data → More → Other → Python script**
2. Enter a Python script such as:

```python
import pandas as pd
df = pd.read_excel('C:/Data/Budget_Outlays.xlsx', sheet_name='FY2024')
```

Power BI executes the code and imports the resulting `pandas.DataFrame` as a table.

### B. Data Cleaning Example

```python
import pandas as pd

df = pd.read_csv('C:/Data/appropriations.csv')

# Clean missing values
df.fillna(0, inplace=True)

# Standardize date formats
df['Date'] = pd.to_datetime(df['Date'])

# Group data by Appropriation Title
grouped = df.groupby('Title')['Obligations'].sum().reset_index()
```

The variable (`grouped`) becomes your transformed dataset.

---

## 📈 4. Python Visuals in Power BI

1. In the report canvas, select **Python visual**.
2. Add fields to the *Values* pane (Power BI passes them as `dataset`).
3. In the script editor, type:

```python
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-darkgrid')
sns.barplot(x='Agency', y='BudgetAuthority', data=dataset)
plt.xticks(rotation=45)
plt.title('Budget Authority by Agency (FY2024)')
```

⚠️ *Python visuals are static (PNG), not interactive.*

---

## 🧠 5. Machine Learning in Power BI

Integrate `scikit-learn` models directly in visuals or preprocessing scripts.

### Example — Forecasting Budget Outlays

```python
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

X = dataset[['FiscalYear']]
y = dataset['Outlays']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = LinearRegression().fit(X_train, y_train)
pred = model.predict(X_test)

r2 = r2_score(y_test, pred)

plt.plot(X_test, y_test, 'o', label='Actual')
plt.plot(X_test, pred, 'r-', label='Predicted')
plt.title(f'Outlay Forecast (R²={r2:.3f})')
plt.legend()
```

---

## 🧮 6. Common Python Workflows in Power BI

| Task           | Library                     | Integration Layer     |
| -------------- | --------------------------- | --------------------- |
| Data Wrangling | pandas, numpy               | Power Query           |
| Forecasting    | prophet, statsmodels        | Power Query or Visual |
| Clustering     | scikit-learn                | Power Query           |
| Text Analysis  | nltk, spacy                 | Power Query           |
| Custom Plots   | matplotlib, seaborn, plotly | Python Visual         |

---

## 📊 7. Dashboards in Power BI

### A. Structure

Power BI visualization has three main presentation layers:

| Layer             | Description                          | Audience     |
| ----------------- | ------------------------------------ | ------------ |
| **Report**        | Multi-page analytical canvas         | Analysts     |
| **Dashboard**     | One-page summary pinned from reports | Executives   |
| **App/Workspace** | Group of reports/dashboards          | Organization |

### B. Build Process

1. **Design visuals** in Power BI Desktop.
2. **Publish** to Power BI Service.
3. **Pin visuals** to dashboards using 📌.
4. **Arrange layout**: KPI cards → Charts → Forecasts → Details.

---

## 📈 8. Dashboard Visual Types

| Type           | Example                             | Description       |
| -------------- | ----------------------------------- | ----------------- |
| **Bar/Column** | Obligations by Agency               | Quick comparisons |
| **Line/Area**  | Outlays over time                   | Trend analysis    |
| **Scatter**    | Budget vs. Execution                | Correlation       |
| **Map**        | Geographic funding                  | Regional impact   |
| **Python**     | Regression plot, correlation matrix | Advanced analysis |

### Example — Regression Plot in Dashboard

```python
import seaborn as sns
import matplotlib.pyplot as plt

sns.regplot(data=dataset, x='FiscalYear', y='Outlays', ci=None, color='navy')
plt.title('Outlay Regression (FY2010–FY2024)')
plt.xlabel('Fiscal Year')
plt.ylabel('Outlays ($B)')
```

---

## 🧭 9. Dashboard Interaction Features

| Feature                  | Function                    | Example                        |
| ------------------------ | --------------------------- | ------------------------------ |
| **Cross-filtering**      | Click → filters all visuals | Select “Army” to filter        |
| **Drillthrough**         | Detail navigation           | Program → Project              |
| **Alerts**               | Threshold notifications     | “Unobligated Balance > $100M”  |
| **Natural Language Q&A** | Auto-visualization          | Type “Total Outlays by Agency” |
| **Bookmarks**            | Save visual states          | Compare baseline vs. scenario  |

---

## 🎨 10. Dashboard Design Principles

| Principle        | Implementation              | Example                         |
| ---------------- | --------------------------- | ------------------------------- |
| **Hierarchy**    | KPIs → Trends → Details     | Show high-level first           |
| **Clarity**      | Avoid 3D/decoration         | Neutral palette                 |
| **Context**      | Show units and fiscal scope | “Outlays (in billions, FY2024)” |
| **Simplicity**   | Unified filters             | One Agency slicer               |
| **Storytelling** | Combine ML visuals          | Forecast beside actuals         |

---

## 🧩 11. Python Dashboard Enhancements

### A. Statistical Forecasting

```python
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt

model = ExponentialSmoothing(dataset['Outlays'], trend='add', seasonal=None)
fit = model.fit()
plt.plot(fit.fittedvalues, label='Fitted')
plt.plot(dataset['Outlays'], 'k--', label='Actual')
plt.title('Smoothed Trend Forecast')
plt.legend()
```

### B. Correlation Heatmap

```python
import seaborn as sns
sns.heatmap(dataset.corr(), annot=True, cmap='coolwarm')
plt.title('Appropriation Correlation Matrix')
```

### C. Word Cloud

```python
from wordcloud import WordCloud
import matplotlib.pyplot as plt

text = ' '.join(dataset['Justification'].astype(str))
cloud = WordCloud(width=800, height=400, background_color='white').generate(text)
plt.imshow(cloud, interpolation='bilinear')
plt.axis('off')
plt.title('Frequent Terms in Budget Justifications')
```

---

## ⚙️ 12. Automation — Opening and Closing Power BI Reports with Python

### Example Script

```python
from __future__ import annotations
import time
import subprocess
from pathlib import Path
import psutil

def open_powerbi_report_auto(report_path: str,
                             powerbi_path: str = r"C:\Program Files\Microsoft Power BI Desktop\bin\PBIDesktop.exe",
                             duration: int = 300) -> None:
    """
    Purpose:
    --------
    Opens a Power BI Desktop report (.pbix), waits for a specified duration,
    and closes it automatically — ideal for scheduled refresh jobs.
    """
    report = Path(report_path)
    if not report.exists():
        raise FileNotFoundError(f"Report not found: {report}")

    proc = subprocess.Popen([powerbi_path, str(report)])
    print(f"Opened {report.name}. Waiting {duration}s before closing...")
    time.sleep(duration)

    for p in psutil.process_iter(['pid', 'name']):
        if 'PBIDesktop' in p.info['name']:
            p.terminate()
            try:
                p.wait(timeout=10)
            except psutil.TimeoutExpired:
                p.kill()
    print("Closed Power BI successfully.")

# Example usage:
open_powerbi_report_auto(r"C:\Reports\BudgetAnalytics.pbix", duration=600)
```

---

## 🔧 13. Python Libraries for Power BI Workflows

| Category             | Library                            | Description                               |
| -------------------- | ---------------------------------- | ----------------------------------------- |
| **Core Analytics**   | pandas, numpy                      | Data wrangling and numerical operations   |
| **Machine Learning** | scikit-learn, statsmodels, prophet | Forecasting, regression, clustering       |
| **Visualization**    | matplotlib, seaborn, plotly        | Data visualization                        |
| **NLP**              | nltk, spacy, textblob              | Sentiment and entity extraction           |
| **Automation**       | subprocess, psutil, requests       | Launching, monitoring, API calls          |
| **Integration**      | msal, powerbiclient                | Power BI REST API and Jupyter integration |
| **Connectivity**     | pyodbc, sqlalchemy, openpyxl       | External data sources                     |

---

## 🧠 14. Power BI + Python Architectural Summary

| Layer                | Function                     | Python Role                 |
| -------------------- | ---------------------------- | --------------------------- |
| **Power Query**      | ETL (Extract–Transform–Load) | pandas, numpy               |
| **Data Model**       | DAX + Tabular Analysis       | optional Python preprocess  |
| **Visualization**    | Reports, Dashboards          | matplotlib, seaborn, plotly |
| **Automation**       | Desktop launch, refresh      | psutil, subprocess          |
| **Machine Learning** | Predictive analytics         | scikit-learn, statsmodels   |
| **NLP / AI**         | Text interpretation          | nltk, spacy, transformers   |
| **External API**     | REST + Azure Integration     | requests, msal              |

---

## 📚 15. Recommended Resources

| Title                                            | Author         | Focus                                     |
| ------------------------------------------------ | -------------- | ----------------------------------------- |
| *Machine Learning with PyTorch and Scikit-Learn* | Raschka et al. | Applied ML & AI                           |
| *Introduction to Machine Learning with Python*   | Müller & Guido | Core concepts & pipelines                 |
| *Excel 2019 Power Programming with VBA*          | Walkenbach     | Complementary automation principles       |
| *Pro WPF 4.5 in C#*                              | MacDonald      | Visualization & UI architecture analogies |

---

## ✅ 16. Summary

Power BI + Python integration enables analysts and developers to:

* Seamlessly **automate ETL** through Python scripts in Power Query.
* Build **statistical and machine learning models** directly in Power BI.
* Create **highly customized visuals** with matplotlib, seaborn, and plotly.
* Develop **executive dashboards** combining BI and AI-driven insights.
* Automate **report refreshes and distribution** via Python processes.

Together, they transform Power BI into a **full-spectrum data intelligence platform** —
equally capable of traditional business reporting and advanced predictive analytics.

---

**End of Tutorial**
✍️ *Author: Terry D. Eppler*
📧 *[terryeppler@gmail.com](mailto:terryeppler@gmail.com)*

---

Would you like me to format this next as a **multi-cell Jupyter Notebook (.ipynb)** layout with Markdown and code cells separated? That way you can execute section-by-section directly in Jupyter.
